# 🚀 OpenVLA Finetuning on a Custom **LIBERO Spatial**-style Dataset (Colab)

This notebook scaffolds a **from-checkpoint finetune** of **OpenVLA** on your **local custom dataset** targeting the **LIBERO Spatial** benchmark distribution.

It includes:
- GPU + environment setup
- OpenVLA repo install (editable)
- Data mounting (Google Drive) and a minimal **dataset adapter** for LIBERO-style episodes
- Training loop (PyTorch + `accelerate`), optional Weights & Biases logging
- Periodic eval, checkpoint saving, and optional push to Hugging Face Hub

> ⚠️ **Note:** OpenVLA's API paths can change. This template uses the common community layout  
> (`openvla` repo with `prismatic/*` modules). If an import fails, follow the inline TODO to adjust the import path to your cloned repo.

## 0. Runtime
- Runtime → **GPU (T4/A100/V100)**
- Python ≥ **3.10**

In [ ]:
import torch, sys, platform, subprocess, os, time
print('Python:', sys.version)
print('CUDA available:', torch.cuda.is_available())
print('GPU:', torch.cuda.get_device_name(0) if torch.cuda.is_available() else None)
print('Torch:', torch.__version__)

Python: 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
CUDA available: True
GPU: NVIDIA A100-SXM4-40GB
Torch: 2.8.0+cu126


## 1. Install dependencies & clone OpenVLA

In [ ]:
!nvidia-smi


Sat Oct 11 14:55:46 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   30C    P0             45W /  400W |       5MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
# 1) Install condacolab (one time per fresh runtime)
!pip -q install -U condacolab
import condacolab
condacolab.install()   # ⚠️ This will restart the runtime

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:06
🔁 Restarting kernel...


In [ ]:
# 2) After auto-restart, you now have conda+mamba
!mamba --version
!conda --version

# Create environment
!conda create -y -n openvla-env python=3.10

# Update pip
# !conda run -n openvla-env pip install -U pip

print("ok up to here")

# Install PyTorch CUDA 12.4 wheels
!conda run -n openvla-env pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

# Sanity check - use a Python file instead of heredoc
print("Checking PyTorch installation...")
!conda run -n openvla-env python -c "import torch, sys; print('Python:', sys.version); print('Torch:', torch.__version__, '| CUDA:', torch.version.cuda, '| cuda_available:', torch.cuda.is_available()); print('GPU:', torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'N/A')"

# Clone and install openvla
# !conda run -n openvla git clone https://github.com/openvla/openvla.git
# %cd /content/openvla
# !conda run -n openvla pip install -e .



mamba 1.5.12
conda 24.11.2
conda 24.11.2
Channels:
 - conda-forge
Platform: linux-64
Solving environment: / - done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.2
    latest version: 25.9.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local/envs/openvla-env

  added / updated specs:
    - python=3.10


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bzip2-1.0.8                |       hda65f42_8         254 KB  conda-forge
    ca-certificates-2025.10.5  |       hbd8a1cb_0         152 KB  conda-forge
    ld_impl_linux-64-2.44      |       ha97dd6f_2         730 KB  conda-forge
    libexpat-2.7.1             |       hecca717_0          73 KB  conda-forge
    libffi-3.4.6               |       h2dba641_1          56 KB  conda-forge
    libgcc-15.2.0              |     

In [ ]:
!git clone https://github.com/808kalli/openvla_franka.git /content/openvla
%cd /content/openvla
!ls
!conda run -n openvla-env pip install -e .
# Set environment variable
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

Cloning into '/content/openvla'...
remote: Enumerating objects: 653, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 653 (delta 59), reused 70 (delta 54), pack-reused 564 (from 2)
Receiving objects: 100% (653/653), 334.65 KiB | 4.29 MiB/s, done.
Resolving deltas: 100% (349/349), done.
/content/openvla
'=0.20.0'	  Makefile		    requirements-min.txt
'=0.21.0'	  --model_family	    scripts
'=0.25.0'	  --num_trials_per_task     --seed
 --center_crop	  --pretrained_checkpoint   --task_suite_name
 experiments	  prismatic		    vla-scripts
 LICENSE	  pyproject.toml
 --load_in_8bit   README.md
Obtaining file:///content/openvla
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requ

## 3. Upload dataset


In [ ]:
# ============================================
# Mount Google Drive (better for large datasets)
# ============================================
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# Git Pull

In [ ]:
!git pull origin main


From https://github.com/808kalli/openvla_franka
 * branch            main       -> FETCH_HEAD
Already up to date.


# 4. Fine-tune

In [ ]:
import wandb

!wandb login

%cd /content/openvla

# # Run with conda env and correct package
# !PYTHONPATH=/content/openvla:$PYTHONPATH \
#   conda run -n openvla-env torchrun --standalone --nnodes 1 --nproc-per-node 1 vla-scripts/finetune.py \
#     --vla_path "openvla/openvla-7b" \
#     --data_root_dir /content/drive/MyDrive/datasets/ \
#     --dataset_name openvla_libero_spatial \
#     --run_root_dir /content/checkpoints_10eps \
#     --adapter_tmp_dir /content/adapter_tmp \
#     --lora_rank 32 \
#     --batch_size 8 \
#     --wandb_entity eliaskallioras-national-technical-university-of-athens \
#     --grad_accumulation_steps 2 \
#     --learning_rate 5e-4 \
#     --image_aug False \
#     --wandb_project openvla-libero-finetune \
#     --save_steps 5000


!PYTHONPATH=/content/openvla:$PYTHONPATH \
  conda run -n openvla-env torchrun --standalone --nnodes 1 --nproc-per-node 1 vla-scripts/finetune.py \
    --vla_path /content/drive/MyDrive/checkpoints/openvla_libero_finetuned \
    --data_root_dir /content/drive/MyDrive/datasets/ \
    --dataset_name openvla_libero_object \
    --run_root_dir /content/checkpoints_object_10eps \
    --adapter_tmp_dir /content/adapter_tmp \
    --lora_rank 32 \
    --batch_size 8 \
    --wandb_entity eliaskallioras-national-technical-university-of-athens \
    --grad_accumulation_steps 2 \
    --learning_rate 5e-4 \
    --image_aug False \
    --wandb_project openvla-libero-finetune \
    --max_steps 20000 \
    --save_steps 10000


Streaming output truncated to the last 5000 lines.
100%|██████████| 20000/20000 [7:56:05<00:00,  1.42s/it]/usr/local/envs/openvla-env/lib/python3.10/site-packages/peft/utils/save_and_load.py:180: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:  25%|██▌       | 1/4 [00:00<00:02,  1.15it/s]

Loading checkpoint shards:  50%|█████     | 2/4 [00:01<00:01,  1.22it/s]

Loading checkpoint shards:  75%|███████▌  | 3/4 [00:02<00:00,  1.30it/s]

Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.51it/s]

                                                       



In [ ]:
import os
import shutil

# Source and destination paths

source_dir = "/content/checkpoints_object_10eps/spatial-to-object-finetuning"
dest_dir = "/content/drive/MyDrive/checkpoints_object_10eps/spatial-to-object-finetuning"

# Create destination directory if it doesn't exist
os.makedirs(dest_dir, exist_ok=True)

# Get all files in the source directory
checkpoint_files = os.listdir(source_dir)

print(f"Found {len(checkpoint_files)} files to copy")

# Copy files one by one
for i, filename in enumerate(checkpoint_files, 1):
    source_file = os.path.join(source_dir, filename)
    dest_file = os.path.join(dest_dir, filename)

    # Only copy files, not subdirectories
    if os.path.isfile(source_file):
        file_size_mb = os.path.getsize(source_file) / 1024 / 1024
        print(f"[{i}/{len(checkpoint_files)}] Copying: {filename} ({file_size_mb:.2f} MB)")
        shutil.copy2(source_file, dest_file)
        print(f"  ✓ Done")
    else:
        print(f"[{i}/{len(checkpoint_files)}] Skipping directory: {filename}")

print("\n✅ All checkpoint files copied to Google Drive!")

Found 19 files to copy
[1/19] Copying: modeling_prismatic.py (0.02 MB)
  ✓ Done
[2/19] Copying: tokenizer_config.json (0.00 MB)
  ✓ Done
[3/19] Copying: model-00003-of-00004.safetensors (4718.22 MB)
  ✓ Done
[4/19] Copying: training_state.pt (416.99 MB)
  ✓ Done
[5/19] Copying: processing_prismatic.py (0.01 MB)
  ✓ Done
[6/19] Copying: configuration_prismatic.py (0.01 MB)
  ✓ Done
[7/19] Copying: dataset_statistics.json (0.00 MB)
  ✓ Done
[8/19] Copying: generation_config.json (0.00 MB)
  ✓ Done
[9/19] Copying: model-00002-of-00004.safetensors (4718.20 MB)
  ✓ Done
[10/19] Copying: special_tokens_map.json (0.00 MB)
  ✓ Done
[11/19] Copying: added_tokens.json (0.00 MB)
  ✓ Done
[12/19] Copying: tokenizer.json (1.76 MB)
  ✓ Done
[13/19] Copying: tokenizer.model (0.48 MB)
  ✓ Done
[14/19] Copying: model-00004-of-00004.safetensors (250.50 MB)
  ✓ Done
[15/19] Copying: model-00001-of-00004.safetensors (4696.96 MB)
  ✓ Done
[16/19] Copying: config.json (0.06 MB)
  ✓ Done
[17/19] Copying: mod

In [ ]:
import time

for i in range(60):
  time.sleep(5)
  print("keeping session alive")